In [ ]:
#!/usr/bin/env python3
import os, platform, subprocess, sys

The goal of this script is to:
* Give a walkthrough of how to setup the Bitcoin node with an RPC server
* Link Python properly to the RPC server
* And test the connection to the Bitcoin node

Step 1: go to https://bitcoin.org/en/full-node#setup-a-full-node and follow the system appropriate instructions on downloading and setting up a full node

Step 2: make sure that the Bitcoin GUI (bitcoin-qt) runs correctly. Paths might need to be set depending on your platform. Libraries may need to be installed as well.

In [ ]:
"""
Finding and importing provided libraries regardless of system and platform type
"""
## find and import libraries relative to this file's location (regardless of script or notebook)
## platfrom specific naming already handled within the lib directory's __init__ file
if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket
from lib.rpcauth  import rpcauth
from lib.sign     import sign_tx

Step 3: setting up the Bitcoin config. For a more in-depth exploration of Bitcoin config options: https://jlopp.github.io/bitcoin-core-config-generator/ is a great resource.
For a basic RPC server setup we will be using, please fill in the variables below and copy the output configutation to your Bitcoin config file (which can be found under Settings->Options->Open Configuration File in bitcoin-qt). 

In [ ]:
'''
Step 3.1: choose a dummy (not real money) network/chain to use.
          While all this code would work with the mainnet or main chain (where the cash is), we
          probably don't want to be demo-ing how to make transactions with real world money. 
          Choose which flavor of monopoly cash you wish to play with.
          
          Options are:
              'test' - which is the test network (need a 'faucet' to get monopoly money)
              'regtest' - which is the regression/private test network (mine your own monopoly money)
              'main' - glhf, where the real money is. Also the default if no chain is specified.
'''
chain = "test"
if chain not in ["test", "regtest", "main"]:
    raise Exception(chain, "not one of the accepted options of: test, regtest, main")

In [ ]:
'''
Step 3.2: choose where your Bitcoin node is hosted. Yes, this means IP address.
          If your node is on the same host as this script, then this step is real easy.
          Also, please choose a free port where your node is hosted for the RPC server to listen to.
'''
nodeIP = "127.0.0.1"
nodePort = "18444"

def ping(IP):
    param = '-n' if platform.system().lower()=='windows' else '-c'
    command = ['ping', param, '1', IP]
    return subprocess.call(command) == 0

if not ping(nodeIP):
    raise Exception(nodeIP, " is not a reachable address, please try again")

In [ ]:
'''
Step 3.3: choose a username and password
'''
username = "user"
password = "password"

if username == "" or password == "":
    raise Exception("username and password may not be empty strings")

In [ ]:
'''
Step 3.4: copy and paste in the generated bitcoin.conf file below
          After copying in the new configurations, bitcoin-qt will need to be restarted
'''

print('chain=',chain, sep='')
print('daemon=1')

print('\n[',chain,']', sep='')
print('server=1')
print('rpcbind=',nodeIP, sep='')
print('rpcauth=',rpcauth(username, password), sep='')
print('rpcport=',nodePort, sep='')
print('rpcallowip=0.0.0.0/0')


Step 4: once Bitcoin is configured correctly, we will need a wallet to stash funds in.
        Create a wallet in bitcoin-qt (File->Create Wallet) or use an existing one (File->Open Wallet)
        We will also need to have funds added to the wallet.

In [ ]:
'''
Step 4.1: add wallet name to this script
'''
wallet = "test2"

if wallet == "":
    raise Exception("wallet name may not be an empty string")

In [ ]:
'''
Step 4.2: setup our RPC socket using the existing information
'''
rpc = RpcSocket({ 'wallet': wallet,
                  'username': username,
                  'password': password,
                  'url' : nodeIP,
                  'port': nodePort})
assert rpc.check()

In [ ]:
'''
Step 4.3: fund wallet depending on chain type
'''
if "newReceivingAddress" not in locals() or newReceivingAddress == "":
    newReceivingAddress = rpc.call("getnewaddress")
    
if chain == "test":
    print("Use the receiving address", newReceivingAddress, "to receive funds from a testnet faucet")
    print("Some example faucets are listed at: https://en.bitcoin.it/wiki/Testnet")
    print("Once sent, please wait for transaction to be confirmed before proceeding")
elif chain == "regtest":
    blocks = 10
    print("Mining", blocks, "to", newReceivingAddress)
    rpc.call("generatetoaddress", [blocks, newReceivingAddress])
else:
    print("glhf: you must use real money to get bitcoins to ", newReceivingAddress)

In [ ]:
if float(rpc.call("getbalance")) == 0:
    raise Exception("go back one step")
else:
    print("You have a balance of:", rpc.call("getbalance"), "BTC")

Now Bitcoin nodes, RPC server, and wallet should be set up and stocked with funds.
You are ready move on to the next script and make a transaction!

In [ ]:
'''
Step ???: for the next example scripts, all the variables you have chosen for your RPC server setup 
'''
print('nodeIP = \"',nodeIP,'\"', sep='')
print('nodePort = \"',nodePort,'\"', sep='')
print('username = \"',username,'\"', sep='')
print('password = \"',password,'\"', sep='')
print('wallet = \"',wallet,'\"', sep='')